In [1]:
!pip install langchain


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "<paste-your-openai-key>"
model = ChatOpenAI(model="gpt-4.1")

In [5]:
from langchain.agents import AgentState
from typing_extensions import NotRequired
from typing import Literal

SupportStep = Literal["warranty_collector" , "issue_classifier", "resolution_specialist"]

class SupportState(AgentState):
    """State for customer support workflow."""
    current_step: NotRequired[SupportStep]
    warranty_status: NotRequired[Literal["in_warranty", "out_of_warranty"]]
    issue_type: NotRequired[Literal["hardware","software"]]


In [13]:
from langchain.tools import tool, ToolRuntime
from langchain.messages import ToolMessage
from langgraph.types import Command

@tool
def record_warranty_status(
    status: Literal["in_warranty","out_of_warranty"],
    runtime: ToolRuntime[None, SupportState],
) -> Command:
    """Record the customer's warranty status and transition to issue classification"""
    return Command(
        update={
            "messages": [
                ToolMessage(
                    content=f"Warranty status recorded as: {status}",
                    tool_call_id=runtime.tool_call_id,
                )
            ],
            "warranty_status": status,
            "current_step": "issue_classifier",
        }
    )

@tool
def record_issue_type(
    issue_type: Literal["hardware","software"],
    runtime: ToolRuntime[None, SupportState],
) -> Command:
    """Record the type of issues and trasition to resolution specialist."""
    return Command(
        update={
            "messages" : [
                ToolMessage(
                    content=f"issue type recorded as: {issue_type}",
                    tool_call_id=runtime.tool_call_id,
                )
            ],
            "issue_type": issue_type,
            "current_step": "resolution_specialist",
        }
    )

@tool
def esclate_to_human(reason: str) -> str:
    """Esclate the case to a human support specialist."""
    return f"Esclating to human support. Reason: {reason}"

@tool
def provide_resolution(solution: str) -> str:
    """Provide a solution to the customer's issue."""
    return f"Solution provided: {solution}"

In [14]:
# Define prompts as constants for easy reference
WARRANTY_COLLECTOR_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Warranty verification

At this step, you need to:
1. Greet the customer warmly
2. Ask if their device is under warranty
3. Use record_warranty_status to record their response and move to the next step

Be conversational and friendly. Don't ask multiple questions at once."""

ISSUE_CLASSIFIER_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Issue classification
CUSTOMER INFO: Warranty status is {warranty_status}

At this step, you need to:
1. Ask the customer to describe their issue
2. Determine if it's a hardware issue (physical damage, broken parts) or software issue (app crashes, performance)
3. Use record_issue_type to record the classification and move to the next step

If unclear, ask clarifying questions before classifying."""

RESOLUTION_SPECIALIST_PROMPT = """You are a customer support agent helping with device issues.

CURRENT STAGE: Resolution
CUSTOMER INFO: Warranty status is {warranty_status}, issue type is {issue_type}

At this step, you need to:
1. For SOFTWARE issues: provide troubleshooting steps using provide_solution
2. For HARDWARE issues:
   - If IN WARRANTY: explain warranty repair process using provide_solution
   - If OUT OF WARRANTY: escalate_to_human for paid repair options

Be specific and helpful in your solutions."""

In [15]:
STEP_CONFIG = {
    "waranty_collector": {
        "prompt" : WARRANTY_COLLECTOR_PROMPT,
        "tools": [record_warranty_status],
        "requires": [],
    },
    "issue_classifier": {
        "prompt" : ISSUE_CLASSIFIER_PROMPT,
        "tools": [record_issue_type],
        "requires": ["warranty_status"],
    },
    "resolution_specialist": {
        "prompt": RESOLUTION_SPECIALIST_PROMPT,
        "tools": [provide_resolution, esclate_to_human],
        "requires": ["warranty_status","issue_type"],
    },
}

In [16]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def apply_step_config(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelResponse:
    """Configure agent behaviour based on the current step."""
    current_step = request.state.get("current_step", "warranty_collector")

    stage_config = STEP_CONFIG[current_step]

    for key in stage_config["requires"]:
        if request.state.get(key) is None:
            raise ValueError(f"{key} must be set before reaching {current_step}")

    system_prompt = stage_config["prompt"].format(**request.state)

    request = request.override(
        system_prompt=system_prompt,
        tools=stage_config["tools"],
    )

    return handler(request)

In [17]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

all_tools = [
    record_warranty_status,
    record_issue_type,
    provide_resolution,
    esclate_to_human,
]

agent = create_agent(
    model,
    tools=all_tools,
    state_schema=SupportState,
    checkpointer=InMemorySaver()
)

In [18]:
from langchain.messages import HumanMessage
import uuid

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

print("====Turn 1: Warranty Collection ====")
result = agent.invoke(
    {"messages": [HumanMessage("Hi My phone screen is cracked")]},
    config
)

for msg in result["messages"]:
    msg.pretty_print()


print("\n=== Turn 2: Warranty Response ===")
result = agent.invoke(
    {"messages": [HumanMessage("yes, it's still under warranty")]},
    config
)

for msg in result["messages"]:
    msg.pretty_print()
print(f"Current Step: {result.get("current_step")}")


print("\n=== Turn 3: Issue Description ===")
result = agent.invoke(
    {"messages": [HumanMessage("The screen is physically cracked from dropping it")]},
    config
)

for msg in result["messages"]:
    msg.pretty_print()
print(f"Current Step: {result.get("current_step")}")

print("\n=== Turn 4: Resolution ===")
result = agent.invoke(
    {"messages": [HumanMessage("What should I do?")]},
    config
)

for msg in result["messages"]:
    msg.pretty_print()

====Turn 1: Warranty Collection ====
================================ Human Message =================================

Hi My phone screen is cracked
================================== Ai Message ==================================

I'm sorry to hear that your phone screen is cracked. I can help you with the next steps. First, could you please let me know if your phone is still under warranty? This will help determine the best solution for your situation.

=== Turn 2: Warranty Response ===
================================ Human Message =================================

Hi My phone screen is cracked
================================== Ai Message ==================================

I'm sorry to hear that your phone screen is cracked. I can help you with the next steps. First, could you please let me know if your phone is still under warranty? This will help determine the best solution for your situation.
================================ Human Message =================================

yes,